In [2]:
!pip install streamlit jupyter-server-proxy

In [1]:
pip install openai==1.61.1

     ------------------------------------ 463.1/463.1 kB 239.5 kB/s eta 0:00:00
     -------------------------------------- 208.4/208.4 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
%%writefile app.py
import streamlit as st
from openai import OpenAI

import os
os.environ['OPENAI_API_KEY'] = 'sk-proj-Key'

# Simulated user credentials (Replace with real authentication)
VALID_CREDENTIALS = {"admin": "password123", "user": "test123"}

# Initialize OpenAI client
client = OpenAI()

# Session state for authentication
if "authenticated" not in st.session_state:
    st.session_state.authenticated = False

# Sidebar - Authentication
st.sidebar.title("Login")
username = st.sidebar.text_input("User:")
password = st.sidebar.text_input("Password:", type="password", key="password", help="Enter your password securely")
login_button = st.sidebar.button("Login")

if login_button:
    if username in VALID_CREDENTIALS and VALID_CREDENTIALS[username] == password:
        st.session_state.authenticated = True
        st.sidebar.success("User authenticated successfully.")
    else:
        st.sidebar.error("Invalid username or password.")

# Main Chat UI - Only visible after authentication
if st.session_state.authenticated:
    st.title("Chat with ChatGPT")

    # Initialize conversation history if not present
    if "messages" not in st.session_state:
        st.session_state.messages = [
            {"role": "system", "content": "You are a helpful assistant."}
        ]

    # Display chat history properly
    for msg in st.session_state.messages:
        if msg["role"] == "user":
            with st.chat_message("user"):
                st.write(msg["content"])
        elif msg["role"] == "assistant":
            with st.chat_message("assistant"):
                st.write(msg["content"])

    # User input for chat
    user_input = st.chat_input("Ask a question...")

    if user_input:
        # Show user input in chat UI
        with st.chat_message("user"):
            st.write(user_input)

        # Append user message to session state
        st.session_state.messages.append({"role": "user", "content": user_input})

        # Generate AI response with loading indicator
        with st.chat_message("assistant"):
            with st.spinner("Thinking..."):  # <--- Loading indicator
                completion = client.chat.completions.create(
                    model="gpt-4o",
                    messages=st.session_state.messages,
                    max_tokens=500
                )
                bot_response = completion.choices[0].message.content

            # Display assistant response
            st.write(bot_response)

        # Append assistant response to session state
        st.session_state.messages.append({"role": "assistant", "content": bot_response})
else:
    st.warning("Please log in to access the chat.")


Overwriting app.py


In [4]:
!streamlit run app.py --server.port 8501 --browser.serverAddress="0.0.0.0"

C:\Users\Sai\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2025-03-20 07:57:48.215 Port 8501 is already in use
